## Day 3:  Optimizations/Cython/Running C/C++/Fortran code in Python

In [ ]:
%matplotlib inline
%load_ext Cython

# Inpiration gotten from:
# http://people.duke.edu/~ccc14/sta-663-2016/18D_Cython.html

import multiprocessing as mp
import matplotlib.pylab as plt
import random
import numpy as np
import time

## Try it yourself
Modify the code examples below to speed them up using the methods shown above. First test the example and note the speed on your computer with a comment. Then work on making the code faster by anotating it to see what can use improvement and see if you can speed it up.

### Ex.1) List of primes
This example returns a list of primes between 0 and the max number you put in.

In [ ]:
def is_prime(possiblePrime):
    isPrime = True
    for num in range(2, possiblePrime):
        if possiblePrime % num == 0:
            isPrime = False
    return isPrime


def getPrimes(max_num):
    primes = []
    # Try to modify the for loop to make have it use multiprocessing
    for possiblePrime in range(max_num):
        if is_prime(possiblePrime):
            primes.append(possiblePrime)
            
    return primes

In [ ]:
max_prime = 10000
primes = []
%time primes = getPrimes(max_prime)

# Between 0 and 10000 there are 1229 primes.
len(primes)

In [ ]:
#%load get_primes_mp.py

def getPrimes_mp(max_num):
    print("\033[1m\033[91m\n\nLoad the py file for the answer\n\n\033[0m")
    return []

In [ ]:
max_prime = 10000
primes = []
%time primes = getPrimes_mp(max_prime)

# Between 0 and 10000 there are 1229 primes.
len(primes)

### Ex. 1.1) List of primes Cython
This example returns a list of primes between 0 and the max number you put in.

In [ ]:
#%%cython -a

def is_prime(possiblePrime):
    isPrime = True
    for num in range(2, possiblePrime):
        if possiblePrime % num == 0:
            isPrime = False
    return isPrime


def getPrimes(max_num):
    primes = []
    # Hint: Do we need to loop over every number??
    # What do we know about primes that could help us
    for possiblePrime in range(max_num):
        if is_prime(possiblePrime):
            primes.append(possiblePrime)
            
    return primes

In [ ]:
max_prime = 10000
%time primes = getPrimes(max_prime)

# Between 0 and 10000 there are 1229 primes.
len(primes)

In [ ]:
#%load get_primes.pyx
# ^ Uncomment to find the answer
# NOTE: %%cython has to be the first line so remove the load line as well

def getPrimes_cy(max_num):
    print("\033[1m\033[91m\n\nLoad the py file for the answer\n\n\033[0m")
    return []

In [ ]:
max_prime = 10000

%time primes = getPrimes_cy(max_prime)

# Between 0 and 10000 there are 1229 primes. 
# Make sure you get the same answer all the time.
len(primes)

### Ex. 2) Compute $\pi$ using the formula:


### \begin{equation}
\pi^{2} = 6 \sum_{n=1}^{\infty} \frac{1}{k^{2}}
\end{equation}

In [ ]:
%%cython -a
# Hint: 
# import the c functions instead of using python functions
# from libc.math cimport sqrt
from math import sqrt

# Hint: 
# Functions that are only called in cython can be defined with cdef
# This makes a C function instead of a python function
def recip_square(k):
    den = (k**2)
    return 1/den

def approx_pi(n):
    """Compute an approximate value of pi."""
    val = 0
    for k in range(1, n+1):
        temp = recip_square(k)
        val += temp
    pi = sqrt(6 * val)
    return pi

In [ ]:
%time approx_pi(10000000)

In [ ]:
#%load pi.pyx
# ^ Uncomment to find the answer
# NOTE: %%cython has to be the first line so remove the load line as well

def approx_pi_cy(n):
    print("\033[1m\033[91m\n\nLoad the py file for the answer\n\n\033[0m")
    return 3.0

In [ ]:
%time approx_pi_cy(10000000)

### Ex.3) Mandel fractal drawing

In [ ]:
%%cython -a

cimport cython

## Hints:
# You can import and use C functions in cython
#cdef extern from "complex.h":
#    double cabs(double complex)

def mandel(x, y, max_iters):
    # Use the c complex type
    # cdef double complex c, z
    # c = x + y*1j
    # z = 0.0j
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if z.real*z.real + z.imag*z.imag >= 4:
        # if cabs(z) >= 2:
            return i
    return max_iters

def create_fractal(xmin, xmax, ymin, ymax, image, iters):
    height, width = image.shape

    pixel_size_x = (xmax - xmin)/width
    pixel_size_y = (ymax - ymin)/height

    for x in range(width):
        real = xmin + x*pixel_size_x
        for y in range(height):
            imag = ymin + y*pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x]  = color

In [ ]:
#%load fractal.pyx
# ^ Uncomment to find the answer
# NOTE: %%cython has to be the first line so remove the load line as well

def create_fractal_cython(xmin, xmax, ymin, ymax, gimage, iters):
    print("\033[1m\033[91m\n\nLoad the py file for the answer\n\n\033[0m")

In [ ]:
gimage = np.zeros((1080, 1920), dtype=np.uint32)
xmin, xmax, ymin, ymax = [-2.0, 1.0, -1.0, 1.0]
iters = 100

%time create_fractal_cython(xmin, xmax, ymin, ymax, gimage, iters)

plt.figure(figsize=(15,15))
plt.grid(False)
plt.imshow(gimage, cmap='viridis')
plt.show()

### Ex.4) Speed up wave propogration using cython

See if you can beat the C++/Fortran speed

In [ ]:
%%cython -a

def wave_propogation_fast(num_steps, scale, damping, initial_P, stop_step):
    from math import pi, sin
    omega = 3 / (2 * pi)
    
    size_x = 2 * scale + 1
    size_y = 2 * scale + 1

    # V velocity
    # P presure
    # Initialization
    P = [[0.0 for x in range(size_x)] for y in range(size_y)]
    V = [[[0.0, 0.0, 0.0, 0.0] for x in range(size_x)] for y in range(size_y)]
    P[scale][scale] = initial_P
    for step in range(num_steps):
        if step <= stop_step:
            P[scale][scale] = initial_P * sin(omega * step)
        for i in range(size_y):
            for j in range(size_x):
                V[i][j][0] = V[i][j][0] + P[i][j] - P[i - 1][j] if i > 0 else P[i][j]
                V[i][j][1] = (V[i][j][1] + P[i][j] - P[i][j + 1] if j < size_x - 1 else P[i][j])
                V[i][j][2] = (V[i][j][2] + P[i][j] - P[i + 1][j] if i < size_y - 1 else P[i][j])
                V[i][j][3] = V[i][j][3] + P[i][j] - P[i][j - 1] if j > 0 else P[i][j]

        for i in range(size_y):
            for j in range(size_x):
                P[i][j] -= 0.5 * damping * sum(V[i][j])
    return P

In [ ]:
#%load wave_propogation.pyx
# ^ Uncomment to find the answer
# NOTE: %%cython has to be the first line so remove the load line as well

def wave_propogation_fast(num_steps,scale,damping,initial_P,stop_step):
    print("\033[1m\033[91m\n\nLoad the py file for the answer\n\n\033[0m")
    return np.ones((scale*2,scale*2))

In [ ]:
num_steps = 100
scale = 100
damping= 0.25
initial_P = 250
stop_step = 100

plt.figure(figsize=(10,10))
start = time.time()
pressure = wave_propogation_fast(num_steps,scale,damping,initial_P,stop_step)
stop = time.time()
print(f"{stop - start:.2f} Sec, {num_steps / (stop - start):.2f} Hz")
plt.imshow(pressure,cmap='viridis_r',interpolation='bilinear')